In [ ]:
#
#
#

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from reloading_utils import get_url

In [ ]:
# data/reloading_data - CATRIDGES.csv
cartridges = pd.read_csv(get_url('data/reloading_data - CATRIDGES.csv'))
print(cartridges)

cartridges_to_process = cartridges.loc[:, ['title', 'volume', 'diameter']].to_numpy()
print(cartridges_to_process)

# reloading_data - powder burning rate chart
powder_rates = pd.read_csv(get_url('data/reloading_data - powder burning rate chart.csv'))
print(powder_rates.head())

In [ ]:
def process_data_file(c, v, a):
    filename = f'data/reloading_data - %s.csv' % c
    try:
        d = pd.read_csv(get_url(filename))
        d['cartridge_caliber'] = c
        #d['cartridge_volume'] = v
        #d['cartridge_diameter'] = a
        return d
    except:
        print("ERROR: Cannot process:", filename)
        return None

frames = [ process_data_file(c, v, a) for c, v, a in cartridges_to_process ]
data = pd.concat(frames, ignore_index=True)

# print(data.head())  # DEBUG
print('data.shape', data.shape)

In [ ]:
data[~ np.isnan(data.press_psi)]

In [ ]:
lee = pd.DataFrame(columns=['bul_weight', 'bul_type', *data.columns.to_list()])
# print(101, lee.columns) #DEBUG

bul_weight = None
bul_type = None
for pos in range(0, data.shape[0]):
    line = data.loc[pos].to_numpy()
    # print(123, line) # DEBUG
    if ' grain ' in line[0]: # this is a bullettype "115 grain jacketed"
        bul_weight, bul_type = line[0].split(' grain ')
    else:
        line = (int(bul_weight), bul_type, *line)
        lee.loc[len(lee.index)] = line
print('lee.shape', lee.shape)

In [ ]:
lee

In [ ]:
lee = pd.merge(lee, cartridges, left_on='cartridge_caliber', right_on='title')

In [ ]:
lee.head()

In [ ]:
lee.shape

In [ ]:
#
# CHECK VALIDITY EXAMPLE
#
powders = pd.DataFrame({"key": ["#1", "#2", "#3", "#4", '#5'], "value": [10, 20, 30, 40, 50]})
print(powders)

In [ ]:
data = pd.DataFrame({"powder": ["#1", "#2", "#3", "#4", '#xz'], "load_min": [10, 20, 30, 40, 22 ], 'load_max': [20, 25, 35, 45, 42]})
print(data)

In [ ]:
# INNER JOIN
# SELECT *
# FROM df1
# INNER JOIN df2
#   ON df1.key = df2.key;

pd.merge(powders, data, left_on='key', right_on='powder')

In [ ]:
# LEFT OUTER JOIN
# Show all records from df1.

# SELECT *
# FROM df1
# LEFT OUTER JOIN df2
#  ON df1.key = df2.key;
# pd.merge(df1, df2, on="key", how="left")

left_outer_join = pd.merge(powders, data, left_on='key', right_on='powder', how="left")[['key', 'powder']]
print(left_outer_join)

print(left_outer_join[['key', 'powder']])

# WHERE
# Filtering in SQL is done via a WHERE clause.

# SELECT *
# FROM tips
# WHERE time = 'Dinner';

# tips[tips["total_bill"] > 10]

print(left_outer_join[pd.isna(left_outer_join.powder)])

print('# for loop...')
for i in left_outer_join[pd.isna(left_outer_join.powder)][['key']].to_numpy():
    powder_key = i[0]
    print("Never used in data:", powder_key)


In [ ]:
# RIGHT JOIN
# Show all records from df2.

# SELECT *
# FROM df1
# RIGHT OUTER JOIN df2
#   ON df1.key = df2.key;

right_join = pd.merge(powders, data, left_on='key', right_on='powder', how="right")
print(right_join)

In [ ]:
# WHERE
# Filtering in SQL is done via a WHERE clause.

# SELECT *
# FROM tips
# WHERE time = 'Dinner';

# tips[tips["total_bill"] > 10]



In [ ]:
# Show all powders never used in data tables

print(lee.head())
print(powder_rates.columns.to_list())
print(lee.columns.to_list())

In [ ]:
left_outer_join = pd.merge(powder_rates, lee, on='powder', how="left")[['powder', 'bul_weight']]
print(left_outer_join[left_outer_join.powder == 'CLAYS'])

In [ ]:
for i in left_outer_join[pd.isna(left_outer_join.bul_weight)].to_numpy():
    powder_key = i[0]
    print("Never used in data:", powder_key)

In [ ]:
print(left_outer_join[pd.isna(left_outer_join.bul_weight)])

In [ ]:
# RIGHT JOIN
# Show all records from df2.

# SELECT *
# FROM df1
# RIGHT OUTER JOIN df2
#   ON df1.key = df2.key;

right_join = pd.merge(powder_rates, lee, on='powder', how="right")[['powder', 'rate']]
print(right_join)

right_join[pd.isna(right_join.rate)]

## Verification starts here

In [1]:
#
#
#

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import itertools

from reloading_utils import get_url

In [2]:
lee = pd.read_csv('lee.csv', index_col=0)

In [3]:
lee.head()

,bul_weight,bul_type,powder,s_load,s_velocity,ne_load,ne_velocity,press,units,oal,cartridge_caliber,cartridge_volume,cartridge_diameter,powder_id,bul_type_id,powder_rate
0,115,jacketed,v-N350,5.4,1129.0,6.5,1293.0,NaN,NaN,1.142,9mm parabellum,0.74,0.356,58,0,20
1,115,jacketed,v-3N37,5.6,1129.0,6.5,1289.0,NaN,NaN,1.142,9mm parabellum,0.74,0.356,54,0,19
2,115,jacketed,v-N330,4.5,1076.0,5.4,1227.0,NaN,NaN,1.142,9mm parabellum,0.74,0.356,40,0,15
3,115,jacketed,v-N340,4.8,1129.0,5.4,1220.0,NaN,NaN,1.142,9mm parabellum,0.74,0.356,46,0,18
4,115,jacketed,wSUPER-FLD,4.9,1060.0,5.7,1195.0,31900.0,PSI,1.169,9mm parabellum,0.74,0.356,55,0,19


In [4]:
lee[(lee.bul_weight == 115) & (lee.powder == 'v-N340') ][['s_load', 'cartridge_caliber', 'bul_type']]

,s_load
3,4.8
41,5.0
54,5.2


In [55]:
def s_load(bul_weight, bul_type, powder, cartridge_caliber):
    d = lee[(lee.bul_weight == bul_weight) & 
            (lee.bul_type == bul_type) & 
            (lee.powder == powder) &
            (lee.cartridge_caliber == cartridge_caliber)][['s_load']]
    arr = d.to_numpy()
    # print(arr, arr.shape)
    return arr[0][0]


In [57]:
print(s_load(125, 'lead', 'ACCUR #2', '9mm parabellum'))

IndexError: index 0 is out of bounds for axis 0 with size 0

In [46]:
grp = lee[lee.bul_type == 'XTP'].groupby(['bul_weight', 'bul_type', 'cartridge_caliber'])

In [47]:
for g in grp:
    print(g[0], len(g[1]))

(115, 'XTP', '9mm parabellum') 24
(125, 'XTP', '357 magnum') 28


In [58]:
powder_fractions = pd.DataFrame(columns = ['bul_weight', 'bul_type', 'powder1', 's_load1', 'powder2', 's_load2', 'fraction', 'notes'])
for g in list(grp)[:5]:
    bul_weight, bul_type, cartridge_caliber = g[0]
    # print(bul_weight, bul_type)
    gg = g[1]
    powders = gg['powder'].to_list()
    powder_pairs = itertools.combinations(powders, 2)
    # print(list(powder_pairs))
    for p1, p2 in powder_pairs:
        l1 = s_load(bul_weight, bul_type, p1)
        assert(type(l1) is np.float64)
        l2 = s_load(bul_weight, bul_type, p2)
        assert(type(l2) is np.float64)
        powder_fractions.loc[len(powder_fractions.index)] = [bul_weight, bul_type, p1, l1, p2, l2, l1 / l2]
    

In [59]:
p1, p2, l1

('IMR PB', 'ACCUR #2', 4.5)

In [60]:
g[1]

,bul_weight,bul_type,powder,s_load,s_velocity,ne_load,ne_velocity,press,units,oal,cartridge_caliber,cartridge_volume,cartridge_diameter,powder_id,bul_type_id,powder_rate
129,125,XTP,WIN 296,21.0,1881.0,22.0,1966.0,41400.0,CUP,1.590,357 magnum,1.36,0.358,77,3,26
130,125,XTP,H110,21.0,1881.0,22.0,1966.0,41400.0,CUP,1.590,357 magnum,1.36,0.358,75,3,26
131,125,XTP,H4227,18.0,1692.0,20.0,1839.0,42000.0,CUP,1.590,357 magnum,1.36,0.358,80,3,27
132,125,XTP,v-N110,16.8,1601.0,18.4,1772.0,NaN,NaN,1.575,357 magnum,1.36,0.358,71,3,25
133,125,XTP,R ENFORCER,16.2,1500.0,18.0,1667.0,32570.0,PSI,1.780,357 magnum,1.36,0.358,72,3,25
134,125,XTP,LONGSHORT,8.7,1529.0,9.7,1647.0,42000.0,CUP,1.590,357 magnum,1.36,0.358,62,3,21
135,125,XTP,HS6,9.8,1493.0,10.9,1629.0,42100.0,CUP,1.590,357 magnum,1.36,0.358,43,3,18
136,125,XTP,v-N350,9.6,1496.0,10.2,1561.0,NaN,NaN,1.575,357 magnum,1.36,0.358,58,3,20
137,125,XTP,v-N340,8.6,1444.0,9.3,1517.0,NaN,NaN,1.575,357 magnum,1.36,0.358,46,3,18
138,125,XTP,WIN 231,7.3,1335.0,8.5,1514.0,42700.0,CUP,1.590,357 magnum,1.36,0.358,16,3,7


In [61]:
powder_fractions.shape

(654, 8)

In [62]:
powder_fractions

,bul_weight,bul_type,powder1,s_load1,powder2,s_load2,fraction,notes
0,115,XTP,v-3N37,6.0,v-N340,5.2,1.153846,"(115, 'XTP', '9mm parabellum')"
1,115,XTP,v-3N37,6.0,v-N350,5.9,1.016949,"(115, 'XTP', '9mm parabellum')"
2,115,XTP,v-3N37,6.0,BLUE DOT,7.5,0.800000,"(115, 'XTP', '9mm parabellum')"
3,115,XTP,v-3N37,6.0,v-N330,4.8,1.250000,"(115, 'XTP', '9mm parabellum')"
4,115,XTP,v-3N37,6.0,UNIQUE,5.5,1.090909,"(115, 'XTP', '9mm parabellum')"
...,...,...,...,...,...,...,...,...
649,125,XTP,UNIQUE,8.4,IMR PB,4.5,1.866667,"(125, 'XTP', '357 magnum')"
650,125,XTP,UNIQUE,8.4,ACCUR #2,6.8,1.235294,"(125, 'XTP', '357 magnum')"
651,125,XTP,v-N310,6.0,IMR PB,4.5,1.333333,"(125, 'XTP', '357 magnum')"
652,125,XTP,v-N310,6.0,ACCUR #2,6.8,0.882353,"(125, 'XTP', '357 magnum')"


In [63]:
ppf_table = powder_fractions[['powder1', 'powder2', 'fraction', 'notes']].copy()

In [64]:
ppf_table.sort_values(by=['powder1', 'powder2'])

,powder1,powder2,fraction,notes
608,ACCUR #5,ACCUR #2,1.411765,"(125, 'XTP', '357 magnum')"
600,ACCUR #5,HERC 2400,0.623377,"(125, 'XTP', '357 magnum')"
602,ACCUR #5,IMR 700X,2.133333,"(125, 'XTP', '357 magnum')"
607,ACCUR #5,IMR PB,2.133333,"(125, 'XTP', '357 magnum')"
604,ACCUR #5,POWER PISTOL,1.043478,"(125, 'XTP', '357 magnum')"
...,...,...,...,...
461,v-N350,v-N310,1.600000,"(125, 'XTP', '357 magnum')"
52,v-N350,v-N320,1.475000,"(115, 'XTP', '9mm parabellum')"
458,v-N350,v-N320,1.391304,"(125, 'XTP', '357 magnum')"
46,v-N350,v-N330,1.229167,"(115, 'XTP', '9mm parabellum')"


In [53]:
ppf_table.to_csv('ppf_table.csv')

In [69]:
group_pp = ppf_table.sort_values(by=['powder1', 'powder2']).groupby(['powder1', 'powder2'])
for pp in group_pp:
    if len(pp[1]) > 1:
        # print(pp[0], len(pp[1]))
        print(pp[1])

      powder1   powder2  fraction                           notes
246  AUTOCOMP  IMR 700X  1.307692  (115, 'XTP', '9mm parabellum')
569  AUTOCOMP  IMR 700X  1.666667      (125, 'XTP', '357 magnum')
      powder1 powder2  fraction                           notes
247  AUTOCOMP  IMR PB  1.416667  (115, 'XTP', '9mm parabellum')
574  AUTOCOMP  IMR PB  1.666667      (125, 'XTP', '357 magnum')
      powder1 powder2  fraction                           notes
243  AUTOCOMP  SR7625  1.133333  (115, 'XTP', '9mm parabellum')
568  AUTOCOMP  SR7625  1.293103      (125, 'XTP', '357 magnum')
      powder1    powder2  fraction                           notes
241  AUTOCOMP  UNIVERSAL  1.133333  (115, 'XTP', '9mm parabellum')
563  AUTOCOMP  UNIVERSAL  1.056338      (125, 'XTP', '357 magnum')
        powder1   powder2  fraction                           notes
201  CFE PISTOL  AUTOCOMP  1.039216  (115, 'XTP', '9mm parabellum')
535  CFE PISTOL  AUTOCOMP  0.920000      (125, 'XTP', '357 magnum')
        powde

### Examples of itertools
See also:
https://docs.python.org/3/library/itertools.html

In [ ]:
import itertools
# import pprint

l1 = ['a', 'b', 'c']
l2 = ['X', 'Y', 'Z']

p = itertools.product(l1, l2)

print(p)
# <itertools.product object at 0x1026edd80>

print(type(p))
# <class 'itertools.product'>

In [ ]:
for v in p:
    print(v)

In [ ]:
list(p)

In [ ]:
p = itertools.combinations(l1, 2)

list(p)